In [1]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 4.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_97920694e1374eeb8eab9b0e1dbbc48c.csv"
df=pd.read_csv(url,header=None,na_values=["?"])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,1,1.0,NaN,3.0,2,2,1,2,2,2,2,2.0,2,2,2.0,2,2
1,1,1,1.0,NaN,3.0,2,2,2,2,2,1,2,2.0,2,1,2.0,1,2
2,1,1,2.0,2.0,3.0,1,2,2,2,2,2,2,2.0,2,2,2.0,1,2
3,1,1,2.0,NaN,3.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2
4,1,1,2.0,NaN,3.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2


In [2]:
df.shape

(339, 18)

In [3]:
df.isna().any()

0     False
1     False
2      True
3      True
4      True
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12     True
13    False
14    False
15     True
16    False
17    False
dtype: bool

In [4]:
df.isna().sum()/len(df)

0     0.000000
1     0.000000
2     0.002950
3     0.197640
4     0.457227
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.002950
13    0.000000
14    0.000000
15    0.002950
16    0.000000
17    0.000000
dtype: float64

#### Drop columns 3 and 4. These two columns consist of too many empty cells.

In [5]:
df=df.drop(columns=[3,4])
df.head()

,0,1,2,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,1,1.0,2,2,1,2,2,2,2,2.0,2,2,2.0,2,2
1,1,1,1.0,2,2,2,2,2,1,2,2.0,2,1,2.0,1,2
2,1,1,2.0,1,2,2,2,2,2,2,2.0,2,2,2.0,1,2
3,1,1,2.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2
4,1,1,2.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2


In [6]:
df.iloc[:,[2,10,13]].mode()

,2,12,15
0,2.0,2.0,2.0


In [7]:
df=df.fillna(2)
df.isna().any()

0     False
1     False
2     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
dtype: bool

In [8]:
df=df.rename(columns={0:"Class"})
y=df[["Class"]]
y.groupby("Class")["Class"].count()

Class
1     84
2     20
3      9
4     14
5     39
6      1
7     14
8      6
10     2
11    28
12    16
13     7
14    24
15     2
16     1
17    10
18    29
19     6
20     2
21     1
22    24
Name: Class, dtype: int64

#### The difference classes are the locations of the tumor on a human body. Let's group these organs based on their functions or locations on the body. For example, we are going to group "Duodenum & Sm. Int, Pancreas, Gallbladder, Liver (Classes 6, 11, 12, 13)" together, since these organs are all located in the abdominal area. 

In [9]:
y=y.replace([2,4,10],2)

In [10]:
y=y.replace([3,5],3)

In [11]:
y=y.replace([6,11,12,13],4)

In [12]:
y=y.replace([7,8],5)

In [13]:
y=y.replace([14,15],6)

In [14]:
y=y.replace([16,17],7)

In [15]:
y=y.replace([18,19,20,21],8)

In [16]:
y=y.replace([22,9])

In [17]:
y.groupby("Class")["Class"].count()

Class
1    84
2    36
3    48
4    52
5    20
6    26
7    11
8    62
Name: Class, dtype: int64

In [18]:
X=df.drop(["Class"],axis=1)
X.head()

,1,2,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,1.0,2,2,1,2,2,2,2,2.0,2,2,2.0,2,2
1,1,1.0,2,2,2,2,2,1,2,2.0,2,1,2.0,1,2
2,1,2.0,1,2,2,2,2,2,2,2.0,2,2,2.0,1,2
3,1,2.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2
4,1,2.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2


In [19]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
10,21.830834
1,18.376014
13,12.314183
7,10.084970
12,9.050004
14,7.100808
6,6.353239
2,6.017107
0,4.170356
11,3.559959


In [119]:
X_scores=scores[scores["scores"]>1]
len(X_scores)

14

In [120]:
Xp=X.iloc[:,X_scores.index]
Xp.head()

,13,2,16,10,15,17,9,5,1,14,11,7,12,8
0,2,1.0,2,2,2.0,2,2,2,1,2,2,1,2.0,2
1,2,1.0,1,1,2.0,2,2,2,1,1,2,2,2.0,2
2,2,2.0,1,2,2.0,2,2,1,1,2,2,2,2.0,2
3,2,2.0,1,2,2.0,2,2,1,1,2,2,1,2.0,1
4,2,2.0,1,2,2.0,2,2,1,1,2,2,1,2.0,1


In [121]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(Xp,y,test_size=0.3,random_state=42)
print(X_train.shape)
print(X_test.shape)

(237, 14)
(102, 14)


In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model1=RandomForestClassifier(bootstrap=False,
                              random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model1.predict(X_train)
test_pred=model1.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.8396624472573839
Accuracy for test data:  0.4803921568627451


In [123]:
from sklearn.ensemble import GradientBoostingClassifier
model2=GradientBoostingClassifier(n_estimators=80,learning_rate=0.01,subsample=0.8,
                              random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model2.predict(X_train)
test_pred=model2.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.6244725738396625
Accuracy for test data:  0.47058823529411764


In [124]:
from sklearn import svm
model3=svm.SVC(random_state=0).fit(X_train, np.asarray(y_train).reshape(y_train.shape[0],))
train_pred=model3.predict(X_train)
test_pred=model3.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_train,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.6540084388185654
Accuracy for test data:  0.5392156862745098


#### Let's see if resampling would give the models' performance a boost.

In [125]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(k_neighbors=2,random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [126]:
print(X_res.shape)

(448, 14)


In [148]:
model6=RandomForestClassifier(bootstrap=False,
                              random_state=0).fit(X_res, np.asarray(y_res).reshape(y_res.shape[0],))
train_pred=model6.predict(X_res)
test_pred=model6.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_res,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.8616071428571429
Accuracy for test data:  0.4411764705882353


In [127]:
model2=GradientBoostingClassifier(n_estimators=80,learning_rate=0.01,subsample=0.8,
                              random_state=0).fit(X_res, np.asarray(y_res).reshape(y_res.shape[0],))
train_pred=model2.predict(X_res)
test_pred=model2.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_res,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.6830357142857143
Accuracy for test data:  0.4803921568627451


In [142]:
model5=svm.SVC(random_state=0).fit(X_res, np.asarray(y_res).reshape(y_res.shape[0],))
train_pred=model5.predict(X_res)
test_pred=model5.predict(X_test)
print("Accuracy for train data: ",accuracy_score(y_res,train_pred))
print("Accuracy for test data: ",accuracy_score(y_test,test_pred))

Accuracy for train data:  0.7901785714285714
Accuracy for test data:  0.5490196078431373


#### Resampling the train data deteriorated the RandomForestClassifier's performance on test data, but improved the GradientBoostingClassifier and Support Vector Machine's performances on test data both by 1%.